# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

This notebook, based on a DLC demo, contains the necessary steps to initiate a project.

This shows the basic code, but many of the functions have additional features, please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019. https://www.nature.com/articles/s41596-019-0176-0 or the pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf.

This notebook is to:
- create a project
- extract training frames
- label the frames
- plot the labeled images

Assuming that the training will be done on a different computer, such as an institutional cluster, the following steps are in different files as python scripts.
- create a training set
- train a network
- evaluate a network
- analyze videos
- create automatically labeled videos
- save plots of the trajectories

## Create a new project

It is always good idea to keep the projects seperate if you want to use different networks to analze your data. You should use one project if you are tracking similar subjects/items even if in different environments. This function creates a new project with sub-directories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos (for lableing more data) to the project at any stage of the project. 

In [ ]:
import deeplabcut

In [ ]:
import os
# from pathlib import Path # Is this necessary?

In [ ]:
# If you're coming back to labeling after the project has been set up already, you just need to enter the path.
# If it's a new project from scratch, jump to the next cell.
source_videos_folder = input("Enter the path to the project. ")
print('We are moving to',source_videos_folder)
os.chdir(source_videos_folder)

path_config_file=os.path.join(os.getcwd(),'config.yaml')
print('The project config file is',path_config_file)

In [ ]:
# If you've already started the project you don't need this.
source_videos_folder = input("Enter the path to the source videos. ")
print('We are moving to',source_videos_folder)
os.chdir(source_videos_folder)
print(os.getcwd()) # Remind yourself where you're located.

# Let's define a project from scratch.
# I have a bunch of videos in the folder ...sample_videos. 
# I'll point dlc there and give it some extra params.
# This is for your original videos. Nothing will be changed there.
task='stepclap' # Enter the name of your project
experimenter='CC' # Enter the name of the experimenter
# videos OR FOLDER you want to grab frames from.
# I couldn't make it go over the video files in a folder, always had to specify each video file name.
video=[
    'VID_20210917_133448.mp4',
    'VID_20210917_133508.mp4']

# After initiating, these files will be copied to the project's own folder, one step up in the folder tree.
# This redundant and wastes space but makes things easier if you need to try multiple analysis versions and
# for moving part of the analysis to a different computer. Take the new project folder elsewhere if needed.

path_config_file=deeplabcut.create_new_project(task,experimenter,video,copy_videos=True)
# NOTE: The function returns the path, where your project is located. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)

## Now, go edit the config.yaml file that was created! 
Add your body part labels, edit the number of frames to extract per video, etc. 

#### Note that you can see more information about ANY function by adding a ? at the end,  i.e. 

In [ ]:
deeplabcut.extract_frames?

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) ('uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [ ]:
%matplotlib inline
#there are other ways to grab frames, such as uniformly; please see the paper.

#AUTOMATIC:
deeplabcut.extract_frames(path_config_file) 

In [ ]:
#AND/OR:
#SELECT RARE EVENTS MANUALLY:
%gui wx
deeplabcut.extract_frames(path_config_file,'manual')

## Label the extracted frames

Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [ ]:
# The UI gets stuck a lot. If you did something in a pop-up window, chances you need to restart the kernel. 
# In such case, you only need to tell the python environment the path to the project.
%gui wx
deeplabcut.label_frames(path_config_file)

## Check the labels

[OPTIONAL] Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [ ]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

If the labels need adjusted, you can use relaunch the labeling GUI to move them around, save, and re-plot!

## End of preparations
Stop working here and move the project to the machine that will do the heavy-lifting.

The code below is from the original demo, with small changes, and is kept here for reference and general guidance. 
I just commented it out as a reminder that it's not the most recent.

## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [ ]:
# deeplabcut.create_training_dataset(path_config_file, )
# remember, there are several networks you can pick, the default is resnet-50!

## Start training
This function trains the network for a specific shuffle of the training dataset.

In [ ]:
# deeplabcut.train_network(path_config_file)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
# deeplabcut.evaluate_network(path_config_file, plotting=True)

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
# path_config_folder=os.path.join(os.getcwd(),'stepclap-CC-2021-09-20\\')
# print(path_config_folder)

#Enter a folder OR a list of videos to analyze.
# videofile_path = [os.path.join(path_config_folder,'videos\\VID_20210917_133448.mp4'),os.path.join(path_config_folder,'videos\\VID_20210917_133508.mp4')]
# deeplabcut.analyze_videos(path_config_file,videofile_path)

## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [ ]:
# deeplabcut.extract_outlier_frames?

In [ ]:
# deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [ ]:
#%gui wx
#deeplabcut.refine_labels(path_config_file)

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [ ]:
#NOW, merge this with your original data:
#deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [ ]:
#deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [ ]:
#deeplabcut.create_labeled_video?

In [ ]:
#videofile_path = [
#    os.path.join(path_config_folder,'videos\\VID_20210917_133448.mp4'),
#    os.path.join(path_config_folder,'videos\\VID_20210917_133508.mp4')]
#print(videofile_path)

In [ ]:
#deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
#for making interactive plots.
#%matplotlib notebook 
#deeplabcut.plot_trajectories(path_config_file,videofile_path)

In [ ]:
#or just save to disk
#deeplabcut.plot_trajectories(path_config_file,videofile_path)